# Testing for COVID-19

This is an application of Bayes rule on conditional probability. However, we are not applying Bayes rule directly, but we a using a simulation to virtually test the whole German population for COVID-19. Finally, we want to compute the probability, that is the relative frequency, of being infected if someone is tested positive.

Here are the assumptions:
-  The infection rate in Germany is estimated as the ratio of current infections to the total number of German citizen. Note that we neglect the possibility of undicovered infections.
-  The sensitivity of the COVID-19 test is 93.8%. This is the rate of true positiv results, that is getting a postive result if the person is actually infected.
-  The specificity of the test is 95.6%. This is the rate of true negative results, i.e. getting a negative result if the person is not infected.

In [1]:
import numpy as np
import scipy.stats as st

sensitivity = 0.938
specificity = 0.956

infection_rate = 125e3 / 8e7

# Simple implementation

We implement a test for a single person as the function `test()`. First we determin if the person is actually infected or not. We do this by drawing a random number from a [Bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution). We can use the Bernoulli distribution to simulate the outcome of an experiment which has only two possible outcomes with known probabilities.

Depending on the infection state of the person, we determine the test result. We use either the sensitivity as probability for a positive test if the person is infected or 1 minus the specificity (since we are looking for positive test results).

We return both the infection status as well as the test result.

In [2]:
def test():
    infected = st.bernoulli.rvs(p=infection_rate)
    if infected:
        positive = st.bernoulli.rvs(p=sensitivity)
    else:
        positive = st.bernoulli.rvs(p=(1 - specificity))
    return [infected, positive]

Now, we are setting up the simulation and test 1000000 persons.

In [3]:
%%time
results = []

for i in range(100000):
    results.append(test())

results = np.array(results)

CPU times: user 6.48 s, sys: 186 ms, total: 6.66 s
Wall time: 6.52 s


We want to compute the probability of being infected when tested positive. To select only the positively tested persons, we create a boolean index array. We then sum over the first column of the selection to count the infected persons. The ratio of that number to the number of all positive test is the probability of being infected if tested positive.

In [4]:
positive = results[:, 1]==1
n_positive_and_infected = results[positive, 0].sum()

p_positive_and_infected = n_positive_and_infected / positive.sum()

print("Probability of being infected if tested positive: {:.2f}%".format(100 * p_positive_and_infected))

Probability of being infected if tested positive: 3.17%


In [5]:
print("Probability of being infected: {:.2f}%".format(
    100 * results[:, 0].sum() / results.shape[0]
))
print("Probability of being infected if tested negative: {:.2f}%".format(
    100 * results[~positive, 0].sum() / (~positive).sum()
))

Probability of being infected: 0.15%
Probability of being infected if tested negative: 0.01%


# Computational efficient implementation
The above implementation of the `test()` function is not very efficient because this function is called for each person individually.
It is much more efficient to draw larger sets of random numbers at once. This is achived in the implementation below.

In [6]:
chunk_size = 10000

def test():
    infected = st.bernoulli.rvs(p=infection_rate, size=chunk_size)
    positive = np.where(
        infected == 1,
        st.bernoulli.rvs(p=sensitivity, size=chunk_size),
        st.bernoulli.rvs(p=(1 - specificity), size=chunk_size)
    )
    
    # Here we convert the data to boolean (True or False) to reduce the memory footprint
    return np.vstack((infected, positive)).astype(np.bool)

In [7]:
%%time
results = []

# do not forget to make your results reproducible
np.random.seed(0)

for i in range(int(8e7//chunk_size)):
    test_result = test()
    results.append(test_result)

results = np.concatenate(results, axis=1)

CPU times: user 5.18 s, sys: 118 ms, total: 5.3 s
Wall time: 5.31 s


Now, we can actually test the whole population in a couple of seconds

In [8]:
# we can use boolean arrays to index (select) rows or columns
positive = results[1]
n_positive_and_infected = results[0, positive].sum()

p_positive_and_infected = n_positive_and_infected / positive.sum()
print("Probability of being infected if tested positive: {:.2f}%".format(100 * p_positive_and_infected))

Probability of being infected if tested positive: 3.22%


In [9]:
print("Probability of being infected: {:.2f}%".format(
    100 * results[0].sum() / results.shape[1]
))
n_negative_and_infected = results[0, ~positive].sum()
print("Probability of being infected if tested negative: {:.2f}%".format(
    100 * n_negative_and_infected / (~positive).sum()
))

Probability of being infected: 0.16%
Probability of being infected if tested negative: 0.01%


# Check using Bayes rule

Bayes rule states that

$$ P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

Here, A is the event of being infected and B is the event of being tested positve. Hence P(A|B) is the probability of being infected if tested positive. P(B|A) is the sensitivity of the COVID-19 test and P(A) is the infection rate of the population.

However, the probability of being tested positive P(B) is not readily known. But we can compute the probabilty as the sum of the probabilities of being tested positive and being infected and being tested positive and not being infected.

$$P(B) = P(A \cap B) + P(\overline{A} \cap B) = P(A)P(B|A) + P(\overline{A})P(B|\overline{A})$$

The probability of being tested positive while not being infected is 1 minus the specificity of the test.

$$P(B|\overline{A}) = 1 - P(\overline{B} | \overline{A})$$

Hence we can compute P(B).

In [10]:
p_B = infection_rate * sensitivity + (1 - infection_rate) * (1 - specificity)
print("P(B) = {:.4f}".format(p_B))

P(B) = 0.0454


In [11]:
p_A_under_B = sensitivity * infection_rate / p_B

print("Probability of being infected when tested positive P(A|B) = {:.4f}".format(p_A_under_B))

Probability of being infected when tested positive P(A|B) = 0.0323


Note that the result from using Bayes rule is an exact result and it is the expected outcome of a simulation.
However, the outcome of a particular simulation differs from the exact value because of chance variation while performing the simulation.